In [92]:
import pandas as pd
import numpy as np

# Read in data
codon_usage = pd.read_csv('codon_usage.csv')
codon_usage.head()
codon_usage['DNAtype'].value_counts()
codon_usage['Kingdom'].value_counts()
codon_usage.shape # (13028, 69)
codon_usage['SpeciesID'].nunique() # 12368

/var/folders/_q/_22r26l55_n_rj5snfxxfpn80000gn/T/ipykernel_75964/1951236428.py:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  codon_usage = pd.read_csv('codon_usage.csv')


12368

In [93]:
codon_usage.dtypes
codon_usage.select_dtypes(include=['object']).columns

Index(['Kingdom', 'SpeciesName', 'UUU', 'UUC'], dtype='object')

In [101]:
# clean up column types
codon_usage["UUU"] = pd.to_numeric(codon_usage["UUU"], errors='coerce').dropna()
codon_usage["UUC"] = pd.to_numeric(codon_usage["UUC"], errors='coerce').dropna()

In [95]:
# drop 'SpeciesID', 'SpeciesName'
codon_usage = codon_usage.drop(columns=['SpeciesID', 'SpeciesName'])

In [96]:
# Get one hot encoding of columns 'DNAtype'
one_hot = pd.get_dummies(codon_usage['DNAtype'])
codon_usage = codon_usage.drop('DNAtype', axis=1)
codon_usage = codon_usage.join(one_hot)
codon_usage.head() # (13028, 70)

,Kingdom,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,...,1,2,3,4,5,6,7,9,11,12
0,vrl,1995,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,...,False,False,False,False,False,False,False,False,False,False
1,vrl,1474,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,...,False,False,False,False,False,False,False,False,False,False
2,vrl,4862,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,...,False,False,False,False,False,False,False,False,False,False
3,vrl,1915,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,...,False,False,False,False,False,False,False,False,False,False
4,vrl,22831,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,...,False,False,False,False,False,False,False,False,False,False


In [97]:
codon_usage.dropna(inplace=True)

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(codon_usage.drop('Kingdom', axis=1), codon_usage['Kingdom'], test_size=0.20, random_state=101)

In [99]:
# standardize 'Ncodons'
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_train['Ncodons'].values.reshape(-1,1))
X_train['Ncodons'] = scaler.transform(X_train['Ncodons'].values.reshape(-1,1))
X_test['Ncodons'] = scaler.transform(X_test['Ncodons'].values.reshape(-1,1))
X_test

,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,...,1,2,3,4,5,6,7,9,11,12
5110,105.907308,0.03234,0.01204,0.05359,0.01342,0.01069,0.00207,0.00857,0.00232,0.05183,...,False,False,False,False,False,False,False,False,False,False
292,2.300352,0.02913,0.01961,0.02141,0.03258,0.01947,0.01233,0.01588,0.01874,0.02287,...,False,False,False,False,False,False,False,False,False,False
8106,1.982738,0.00341,0.02627,0.00055,0.00352,0.00325,0.00958,0.00176,0.05859,0.00843,...,False,False,False,False,False,False,False,False,False,False
4886,22.767392,0.05073,0.00770,0.06191,0.01629,0.01678,0.00376,0.00797,0.00124,0.05855,...,False,False,False,False,False,False,False,False,False,False
10537,-0.150795,0.02019,0.05048,0.01739,0.00841,0.03758,0.02580,0.03702,0.02299,0.04262,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10077,0.113275,0.00840,0.07270,0.02100,0.00262,0.01286,0.04514,0.08110,0.01076,0.02100,...,True,False,False,False,False,False,False,False,False,False
4466,-0.251759,0.03175,0.01687,0.02579,0.01190,0.02679,0.00298,0.00496,0.00099,0.04067,...,False,False,False,False,False,False,False,False,False,False
8048,-0.148710,0.00834,0.03724,0.00056,0.00834,0.00834,0.02890,0.00111,0.02946,0.00667,...,False,False,False,False,False,False,False,False,False,False
294,2.873828,0.02804,0.02144,0.01500,0.02448,0.02104,0.01624,0.01340,0.02352,0.02080,...,False,False,False,False,False,False,False,False,False,False


In [100]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('Y_train.csv', index=False)
y_test.to_csv('Y_test.csv', index=False)